# Questions

- Which classroom is the most utilized?
- Which classes go over capacity the most? (cap>0 use remainder to indicate overage)
- Which professors has the most diverse courseload?
- Which days are the most/least popular for classes?
- Which teacher has taught the most students? (via actual, not cap)
- Which classes are attracting less than 10 students? (> 0)

In [ ]:
%load_ext sql
%sql sqlite:///CourseDataWarehouse.db

In [ ]:
%%sql
attach Database 'CourseData.db' AS DB

In [ ]:
%%sql
SELECT *
FROM DB.LOCATIONS
LIMIT 5;

# Create Tables

In [ ]:
%%sql

DROP TABLE IF EXISTS PROGRAMS_DW;
DROP TABLE IF EXISTS PROFESSORS_DW;
DROP TABLE IF EXISTS LOCATIONS_DW;
DROP TABLE IF EXISTS TIMECODES_DW;
DROP TABLE IF EXISTS COURSES_DW;
DROP TABLE IF EXISTS CLASS_FACTS;
DROP TABLE IF EXISTS CLASS_TIMECODES;

In [ ]:
%%sql

CREATE TABLE PROGRAMS_DW (
    Program_id INTEGER PRIMARY KEY,
    program_code TEXT NOT NULL,
    program_name TEXT NOT NULL
);

In [ ]:
%%sql

CREATE TABLE PROFESSORS_DW (
    Professor_id INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

In [ ]:
%%sql

CREATE TABLE LOCATIONS_DW (
    Location_id INTEGER PRIMARY KEY,
    location TEXT NOT NULL
);

In [ ]:
%%sql

CREATE TABLE TIMECODES_DW (
    Timecode_id INTEGER PRIMARY KEY,
    Day TEXT,
    Start TEXT NOT NULL,
    End TEXT NOT NULL
);

In [ ]:
%%sql

CREATE TABLE COURSES_DW (
    Course_id INTEGER PRIMARY KEY, 
    CatalogYear TEXT NOT NULL,
    Catalog_id TEXT NOT NULL,
    Course_title TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Program_id INTEGER,
    FOREIGN KEY (Program_id) REFERENCES PROGRAMS_DW(Program_id)
);

In [ ]:
%%sql

CREATE TABLE CLASS_FACTS_DW (
    Class_id INTEGER PRIMARY KEY,
    Program_id INTEGER,
    Location_id INTEGER,
    Professor_id INTEGER,
    Offering_id INTEGER,
    CatalogYear TEXT NOT NULL,
    Term TEXT NOT NULL,
    Credits REAL,
    Section TEXT NOT NULL,
    Cap INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Remaining INTEGER NOT NULL,
    Timecodes TEXT,
    Count_courses INTEGER,
    Num_students INTEGER,
    Count_location INTEGER,
    FOREIGN KEY (Program_id) REFERENCES PROGRAMS_DW(Program_id),
    FOREIGN KEY (Location_id) REFERENCES LOCATIONS_DW(Location_id),
    FOREIGN KEY (Professor_id) REFERENCES PROFESSORS_DW(Professor_id)   
);

In [ ]:
%%sql

CREATE TABLE CLASS_TIMECODES (
    CT_id INTEGER PRIMARY KEY,
    Class_id INTEGER,
    Timecode_id INTEGER,
    Date_Start TEXT,
    Date_End TEXT NOT NULL,
    FOREIGN KEY (Class_id) REFERENCES CLASS_FACTS_DW(Class_id),
    FOREIGN KEY (Timecode_id) REFERENCES TIMECODES_DW(Timecode_id)
);

# POPULATE TABLES

In [ ]:
%%sql

INSERT INTO PROGRAMS_DW(program_name,program_code)
    SELECT DISTINCT program_name,program_code
    FROM DB.PROGRAMS;
SELECT * FROM PROGRAMS_DW LIMIT 5;

In [ ]:
%%sql

INSERT INTO PROFESSORS_DW(Name)
    SELECT DISTINCT Name
    FROM DB.PROFESSORS;
SELECT * FROM PROFESSORS_DW LIMIT 5;

In [ ]:
%%sql

INSERT INTO LOCATIONS_DW(location)
    SELECT location
    FROM DB.LOCATIONS;
SELECT * FROM LOCATIONS_DW LIMIT 5;

In [ ]:
%%sql

INSERT INTO COURSES_DW(CatalogYear,Catalog_id,Course_title,Credits, Program_id)
    SELECT DISTINCT CatalogYear,Catalog_id,Course_title,Credits, Program_id
    FROM DB.COURSES;
SELECT * FROM COURSES_DW LIMIT 15;

In [ ]:
%%sql
DELETE FROM TIMECODES_DW;
INSERT INTO TIMECODES_DW(Day,Start,End)
    SELECT DISTINCT Day,time(Start) AS StartTime,time(End) AS EndTime
    FROM DB.MEETINGS
    ORDER BY Day, StartTime, EndTime;
SELECT * FROM TIMECODES_DW LIMIT 15;

In [ ]:
%%sql
DROP TABLE IF EXISTS OFFERING_TIMECODE;
CREATE TABLE OFFERING_TIMECODE AS

SELECT DISTINCT Day,time(DB.MEETINGS.Start) AS StartTime,time(DB.MEETINGS.End) AS EndTime, Offering_id,Timecode_id
    FROM DB.MEETINGS
    JOIN DB.COURSE_OFFERINGS USING(Offering_id)
    JOIN TIMECODES_DW USING(Day)
    WHERE TIMECODES_DW.Day = Day AND TIMECODES_DW.Start = StartTime AND TIMECODES_DW.End = EndTime
    ORDER BY Day, StartTime, EndTime;
SELECT * FROM OFFERING_TIMECODE LIMIT 5;

In [ ]:
%%sql

INSERT INTO CLASS_FACTS_DW(Program_id,Location_id,Professor_id,Offering_id,CatalogYear,Term,Credits,Section,Cap,Actual,Remaining,Timecodes)
    SELECT DISTINCT Program_id,Location_id,Professor_id,DB.COURSE_OFFERINGS.Offering_id,COURSES_DW.CatalogYear, DB.COURSE_OFFERINGS.Term, DB.COURSE_OFFERINGS.Credits, DB.COURSE_OFFERINGS.Section, DB.COURSE_OFFERINGS.Cap, DB.COURSE_OFFERINGS.Actual, DB.COURSE_OFFERINGS.Remaining, DB.COURSE_OFFERINGS.Timecodes
    FROM DB.MEETINGS
        JOIN DB.COURSE_OFFERINGS USING (Offering_id)
        JOIN COURSES_DW USING(Course_id)
        JOIN PROGRAMS_DW USING(Program_id)
        JOIN LOCATIONS_DW USING(Location_id)
        JOIN PROFESSORS_DW USING(Professor_id);
        --JOIN OFFERING_TIMECODE USING(Offering_id)
        --JOIN TIMECODES_DW USING(Timecode_id);
SELECT * FROM CLASS_FACTS_DW LIMIT 5;

In [ ]:
%%sql 
DELETE FROM CLASS_TIMECODES; 
INSERT INTO CLASS_TIMECODES(Class_id,Timecode_id,Date_Start,Date_End) 
    SELECT DISTINCT Class_id, Timecode_id,time(DB.MEETINGS.Start) AS Date_Start,time(DB.MEETINGS.End) AS Date_End 
    FROM DB.MEETINGS 
        JOIN TIMECODES_DW ON(TIMECODES_DW.Day = DB.MEETINGS.Day AND TIMECODES_DW.Start = DB.MEETINGS.Start AND TIMECODES_DW.End = DB.MEETINGS.End) 
        JOIN CLASS_FACTS_DW USING (Location_id, Offering_id) 
        ORDER BY Date_Start,Date_End;
SELECT * FROM CLASS_TIMECODES LIMIT 15;

In [ ]:
--DELETE FROM CLASS_TIMECODES; 
--INSERT INTO CLASS_TIMECODES(Class_id,TIMECODES_DW.Timecode_id,Date_Start,Date_End) 
    SELECT DISTINCT Class_id, Timecode_id,time(DB.MEETINGS.Start) AS Date_Start,time(DB.MEETINGS.End) AS Date_End 
    FROM DB.MEETINGS 
        JOIN TIMECODES_DW ON(TIMECODES_DW.Day = DB.MEETINGS.Day AND TIMECODES_DW.Start = DB.MEETINGS.Start AND TIMECODES_DW.End = DB.MEETINGS.End) 
        JOIN CLASS_FACTS_DW USING (Offering_id) 
        ORDER BY Date_Start,Date_End;
--SELECT * FROM CLASS_TIMECODES LIMIT 15;